Install and Import Dependencies

In [32]:
!pip install -r requirements.txt

  Obtaining dependency information for ultralytics==8.2.61 from https://files.pythonhosted.org/packages/ea/3d/860be207d23293037c4cb8fa6e5c659747e87030ca77e3fdfa6c0251c79c/ultralytics-8.2.61-py3-none-any.whl.metadata
  Using cached ultralytics-8.2.61-py3-none-any.whl.metadata (41 kB)
Using cached ultralytics-8.2.61-py3-none-any.whl (804 kB)
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.2.63
    Uninstalling ultralytics-8.2.63:
      Successfully uninstalled ultralytics-8.2.63



[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

Making Custom Dataset

In [36]:
import uuid
import os
import time

In [37]:
IMAGES_PATH = os.path.join('data', 'images')
labels = ['awake', 'drowsy']
number_imgs = 25


In [38]:
cap = cv2.VideoCapture(0)
# Loop through labels
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(10)

    # Loop through image range
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))

        # Webcam feed
        ret, frame = cap.read()

        # Naming out image path
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')

        # Writes out image to file
        cv2.imwrite(imgname, frame)

        # Render to the screen
        cv2.imshow('Image Collection', frame)

        # 2 second delay between captures
        time.sleep(5)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [39]:
!git clone https://github.com/tzutalin/labelImg

fatal: destination path 'labelImg' already exists and is not an empty directory.


In [40]:
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Training Custom Dataset

In [41]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [42]:
!cd yolov5 & pip install -r requirements.txt

KeyboardInterrupt: 

In [46]:
!cd yolov5 && python train.py --img 320 --batch 16 --epochs 200 --data dataset.yml --weights yolov5s.pt --workers 2

train: weights=yolov5s.pt, cfg=, data=dataset.yml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
YOLOv5  v7.0-347-gd6112173 Python-3.12.0 torch-2.3.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, 

Using Trained Model

In [54]:
# dataset 200 epoch 200 img 320 batch 16
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp2/weights/last.pt', force_reload=True)

# dataset 100 epoch 200 img 320 batch 16
# model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp3/weights/last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\chris/.cache\torch\hub\master.zip
YOLOv5  2024-7-25 Python-3.12.0 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [56]:
# Real time object detection

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    result = model(frame)

    cv2.imshow("YOLO", np.squeeze(result.render()))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Integration with ESP32Cam

In [62]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
import paho.mqtt.client as mqtt

# Konfigurasi MQTT
broker = "6b112e44f3c349048a8de67b59c05a69.s1.eu.hivemq.cloud"
port = 8883
topic = "alarm/sleepy"
mqtt_user = "Admin-Kelompok-46"
mqtt_password = "Admin-Kelompok-46"

# Fungsi untuk mengirim sinyal alarm ke MQTT
def send_alarm_signal(client):
    client.publish(topic, "ALARM")

# Callback ketika klien terhubung ke broker
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe(topic)

# Callback ketika pesan diterima dari broker
def on_message(client, userdata, msg):
    print(f"Message received: {msg.payload}")

# Setup klien MQTT
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.username_pw_set(mqtt_user, mqtt_password)
client.tls_set()  # Use TLS for secure connection
client.connect(broker, port, 60)
client.loop_start()  # Start loop to process network traffic and dispatch callbacks

# URL ESP32
url = 'http://192.168.124.239/cam-hi.jpg'

# Load model YOLOv5 custom untuk drowsiness detection
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp3/weights/last.pt', force_reload=True)

def detect_drowsiness(image):
    # Make detections
    results = model(image)
    return results

def some_condition_based_on_results(results):
    # Define your condition here based on YOLOv5 results
    # For example, checking if any detected objects have a specific label
    labels = results.names  # Dictionary of label names
    detected_classes = results.pred[0][:, -1].cpu().numpy()  # Get detected class IDs

    for cls_id in detected_classes:
        if labels[int(cls_id)] == 'drowsiness':  # Replace 'drowsiness' with your class label
            return True
    return False

while True:
    try:
        # Mengambil gambar dari URL ESP32
        img_resp = urllib.request.urlopen(url)
        imgnp = np.array(bytearray(img_resp.read()), dtype=np.uint8)
        frame = cv2.imdecode(imgnp, -1)

        # Horizontal and vertical flip (optional, jika perlu)
        frame = cv2.flip(frame, -1)

        # Deteksi ngantuk
        results = detect_drowsiness(frame)

        # Tampilkan hasil deteksi
        cv2.imshow('ESP-32 CAM Drowsiness Detection', np.squeeze(results.render()))

        # Jika terdeteksi ngantuk, kirim sinyal alarm
        if some_condition_based_on_results(results):
            send_alarm_signal(client)

    except Exception as e:
        print(f'Error: {e}')

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
client.loop_stop()  # Hentikan loop MQTT
client.disconnect()  # Putuskan koneksi MQTT


C:\Users\chris\AppData\Local\Temp\ipykernel_14452\623888533.py:29: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Connected with result code 0


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\chris/.cache\torch\hub\master.zip
YOLOv5  2024-7-25 Python-3.12.0 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


Error: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>
Error: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>
Error: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>
Error: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>
Error: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not prope

KeyboardInterrupt: 

Connected with result code 0
Message received: b'ALARM'
Message received: b'ALARM'
Connected with result code 0
Connected with result code 0
Connected with result code 0
Connected with result code 0
Connected with result code 0
